## Read in the data

In [60]:
import numpy as np
import pandas as pd
import tensorflow
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import fbeta_score

## Training Data

In [2]:
training = pd.read_csv("./converted_data_files/files_and_labels_dataframe.csv")
training['file_path'] = "./train_data/" + training['image_name'] + ".tif"
training.head()

,image_name,labels,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,file_path
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_0.tif
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_1.tif
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_2.tif
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_3.tif
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,./train_data/train_4.tif


In [3]:
print("The complete training dataset contains {} images".format(training.shape[0]))

The complete training dataset contains 40479 images


In [4]:
label_columns = ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation',
                 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy',
                 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming',
                 'selective_logging', 'blow_down']

## Train Test Split

In [24]:
# First, do a train test split to get random subsets of the training data
X_train, X_val, y_train, y_val = train_test_split(training['file_path'], 
                                                  training[label_columns],
                                                  test_size = 0.3,
                                                  random_state = 42)

In [25]:
X_train = training.iloc[X_train.index]
X_train.head()

,image_name,labels,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,file_path
1444,train_1444,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_1444.tif
930,train_930,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_930.tif
13574,train_13574,agriculture clear habitation primary road water,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,./train_data/train_13574.tif
32273,train_32273,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_32273.tif
13462,train_13462,agriculture clear cultivation primary,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,./train_data/train_13462.tif


In [26]:
X_val = training.iloc[X_val.index]
X_val.head()

,image_name,labels,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,file_path
34602,train_34602,clear primary road,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,./train_data/train_34602.tif
11243,train_11243,clear primary water,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,./train_data/train_11243.tif
14499,train_14499,clear conventional_mine habitation primary roa...,0,1,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,./train_data/train_14499.tif
18918,train_18918,cloudy,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,./train_data/train_18918.tif
30631,train_30631,agriculture clear cultivation cultivation habi...,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,./train_data/train_30631.tif


In [8]:
print("The training data has {} images".format(X_train.shape[0]))
print("The validation data has {} images".format(X_val.shape[0]))

The training data has 28335 images
The validation data has 12144 images


# Save the training and validation to CSV

In [9]:
# X_train.to_csv("./converted_data_files/train_train", index=False)
# X_val.to_csv("./converted_data_files/validation", index=False)

## Test data!

In [10]:
# As a reminder, our submission data will have this format.
test_data = pd.read_csv("./kaggle_csv_files/sample_submission_v2.csv")
test_data.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [11]:
# We can drop the "tags" column for now.
test_data.drop(columns = ['tags'], inplace = True)
test_data['file_path'] = test_data['image_name'].map(\
                    lambda x: ("./fixed/" + x + ".tif") if "file_" in x \
                    else ("./test_data/" + x + ".tif"))

In [12]:
# Looks good!
test_data.head()

,image_name,file_path
0,test_0,./test_data/test_0.tif
1,test_1,./test_data/test_1.tif
2,test_2,./test_data/test_2.tif
3,test_3,./test_data/test_3.tif
4,test_4,./test_data/test_4.tif


In [13]:
test_data.tail()

,image_name,file_path
61186,file_9995,./fixed/file_9995.tif
61187,file_9996,./fixed/file_9996.tif
61188,file_9997,./fixed/file_9997.tif
61189,file_9998,./fixed/file_9998.tif
61190,file_9999,./fixed/file_9999.tif


### Predictions to Label Tags

In [14]:
# This function converts our model predictions directly to the format suitable for submission.

# could potentially do threshold list but for now...

def predictions_to_tags(predictions, label_columns, threshold):
    binary_predictions = (predictions > threshold).astype(int) 
    
#     predictions_label = []
#     for prediction in pred_Y:
#     labels = [y_map[i] for i,value in enumerate(prediction) if value > thresholds[i]]
#     predictions_label.append(labels)
    
    prediction_labels_list = []
    labels_map = {index: column for index, column in enumerate(label_columns)}
    
    for image_pred in binary_predictions:
        tags = [labels_map[index] for index, value in enumerate(image_pred) if value == 1]
        prediction_labels_list.append(labels)
    return [' '.join(prediction) for prediction in prediction_labels_list]

# F2 Score

**This is the evaluation metric used in the competition**

In [15]:
# need to separately define own manual fbeta score such that arrays can be passed as tensor in modeling.
# Then, we can use this as a metric in the fit_generator.
def fbeta_score_K(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return result


# Image Data Generators

In [52]:
def train_generator(batch_size,
                    dataframe = training, 
                    x_col = 'file_path', 
                    labels = label_columns):

    
    datagen = ImageDataGenerator(rescale = 1./255.,
                                 horizontal_flip = True,
                                 vertical_flip = True, 
                                 rotation_range = 45,
                                 fill_mode = "nearest",
                                 dtype = "float16")
    
    train_generator = datagen.flow_from_dataframe(
                                dataframe = dataframe,
                                directory = None,
                                x_col = x_col,
                                y_col = labels,
                                batch_size = batch_size,
                                shuffle = True,
                                class_mode = "other",
                                target_size = (256, 256),
                                seed = 42)
    
    return train_generator

def valid_generator(batch_size,
                    dataframe = training, 
                    x_col = 'file_path', 
                    labels = label_columns):
    
    valid_datagen = ImageDataGenerator(rescale = 1./255.,
                                 horizontal_flip = True,
                                 vertical_flip = True, 
                                 rotation_range = 45,
                                 fill_mode = "nearest",
                                 dtype = "float16")
    
    valid_generator = valid_datagen.flow_from_dataframe(
                                    dataframe = dataframe,
                                    directory = None,
                                    x_col = x_col,
                                    y_col = labels,
                                    batch_size = batch_size,
                                    shuffle = False,
                                    class_mode = "other",
                                    target_size = (256, 256),
                                    seed = 42)
    return valid_generator
    

def test_generator(batch_size, dataframe, x_col = 'file_path'):
    test_datagen = ImageDataGenerator(rescale = 1./255.,
                                      dtype = 'float16')

    test_generator = test_datagen.flow_from_dataframe(
                                    dataframe = dataframe,
                                    directory = None,
                                    x_col = x_col,
                                    batch_size = batch_size,
                                    shuffle = False,
                                    class_mode = None,
                                    target_size = (256, 256),
                                    seed = 42)
    return test_generator

# K Fold Cross Validation and Model Testing

In [57]:
n_splits = 5

dataframe = training

x_col = 'file_path'
labels = label_columns

X = training[[x_col]]
y = training[labels]
# model = model

kf = KFold(n_splits=5, shuffle = False, random_state = 42)
folds = kf.split(X = X, y = y)

num_fold = 0
for train_index, val_index in folds:

    X_train = X.iloc[train_index]
    Y_train = y.iloc[train_index]
    X_val = X.iloc[val_index]
    Y_val  = y.iloc[val_index]

    num_fold += 1
    print('Starting KFold number {} of {}\n'.format(num_fold, n_splits))
    
    print('Split train: ', len(X_train), len(Y_train))    
    train_gen = train_generator(dataframe = dataframe.iloc[train_index], 
                                batch_size = 256, x_col= x_col, labels = labels)
    
    print("Train steps per epoch: {}\n".format(train_gen.n//train_gen.batch_size + 1))
    
    print('Split valid: ', len(X_val), len(Y_val))
    val_gen = valid_generator(dataframe = dataframe.iloc[val_index], 
                               batch_size = 256, x_col= x_col, labels = labels)
    
    print("Val steps per epoch: {}\n".format(val_gen.n//val_gen.batch_size + 1))

    
    

Starting KFold number 1 of 5

Split train:  32383 32383
Found 32383 images.
Train steps per epoch: 127

Split valid:  8096 8096
Found 8096 images.
Val steps per epoch: 32

Starting KFold number 2 of 5

Split train:  32383 32383
Found 32383 images.
Train steps per epoch: 127

Split valid:  8096 8096
Found 8096 images.
Val steps per epoch: 32

Starting KFold number 3 of 5

Split train:  32383 32383
Found 32383 images.
Train steps per epoch: 127

Split valid:  8096 8096
Found 8096 images.
Val steps per epoch: 32

Starting KFold number 4 of 5

Split train:  32383 32383
Found 32383 images.
Train steps per epoch: 127

Split valid:  8096 8096
Found 8096 images.
Val steps per epoch: 32

Starting KFold number 5 of 5

Split train:  32384 32384
Found 32384 images.
Train steps per epoch: 127

Split valid:  8095 8095
Found 8095 images.
Val steps per epoch: 32



In [62]:
def KFold_Train(data, model, num_epochs, labels = label_columns, 
                file_path = 'file_path', n_splits = 5, batch_size = 256):
    
    X = data[file_path]
    y = data[labels]
    
    model = model
    history_list = []
    
    kf = KFold(n_splits=5, shuffle = False, random_state = 42)
    folds = kf.split(X = X, y = y)

    num_fold = 0
    for train_index, val_index in folds:

        X_train = X.iloc[train_index]
        Y_train = y.iloc[train_index]
        X_val = X.iloc[val_index]
        Y_val  = y.iloc[val_index]

        num_fold += 1
        print('Starting KFold number {} of {}'.format(num_fold, n_splits))

        print('Split train: ', len(X_train), len(Y_train))    
        train_gen = train_generator(dataframe = dataframe.iloc[train_index], 
                                    batch_size = 256, x_col= x_col, labels = labels)

        print('Split valid: ', len(X_val), len(Y_val))
        val_gen = valid_generator(dataframe = dataframe.iloc[val_index], 
                                   batch_size = 256, x_col= x_col, labels = labels)
        
        model.compile(optimizer = "adam",
              loss='binary_crossentropy',
              metrics=["accuracy", fbeta_score_K])
  
        callbacks = [
                EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint("./saved_val_models/{}_fold_{}".format(str(model), str(num_fold)), 
                                monitor='val_loss', save_best_only=True, verbose=0)]
        
        history = model.fit_generator(train_gen, 
                            steps_per_epoch = (train_gen.n//train_gen.batch_size + 1),
                            validation_data = val_gen,
                            validation_steps = (val_gen.n//val_gen.batch_size + 1),
                            epochs = num_epochs,
                            use_multiprocessing = True,
                            verbose = 1)
        
        history_list.append(history)
        
        
        validation_pred = model.predict_generator(val_gen, 
                                                  steps = (val_gen.n//val_gen.batch_size + 1),
                                                  verbose = 1)
        
        np.save("./kfold_val_preds/{}_val_preds_fold_{}".format(str(model), str(num_fold)))
        
        print(fbeta_score(Y_val, np.array(validation_pred) > 0.2, 
                          beta=2, average='samples'))
        
    return history_list

# Image Data Generator and Image Augmentation

Code derived from this [medium post.](https://medium.com/@vijayabhaskar96/multi-label-image-classification-tutorial-with-keras-imagedatagenerator-cd541f8eaf24)

In [27]:
# datagen = ImageDataGenerator(rescale = 1./255.,
#                              horizontal_flip = True,
#                              vertical_flip = True, 
#                              rotation_range = 45,
#                              fill_mode = "nearest",
#                              dtype = "float16")

# test_datagen = ImageDataGenerator(rescale = 1./255.,
#                                   dtype = 'float16')

In [28]:
# batch_size = 64

# train_generator = datagen.flow_from_dataframe(
#                                 dataframe = X_train[:256],
#                                 directory = None,
#                                 x_col = "file_path",
#                                 y_col = label_columns,
#                                 batch_size = batch_size,
#                                 shuffle = True,
#                                 class_mode = "other",
#                                 target_size = (256, 256),
#                                 seed = 42)

# valid_generator = datagen.flow_from_dataframe(
#                                 dataframe = X_val[:256],
#                                 directory = None,
#                                 x_col = "file_path",
#                                 y_col = label_columns,
#                                 batch_size = batch_size,
#                                 shuffle = False,
#                                 class_mode = "other",
#                                 target_size = (256, 256),
#                                 seed = 42)

# test_generator = test_datagen.flow_from_dataframe(
#                                 dataframe = test_data[:256],
#                                 directory = None,
#                                 x_col = "file_path",
#                                 batch_size = batch_size,
#                                 shuffle = False,
#                                 class_mode = None,
#                                 target_size = (256, 256),
#                                 seed = 42)

Found 256 images.
Found 256 images.
Found 256 images.


### Simple Sequential CNN

In [66]:
def simple_cnn(input_shape = (256, 256, 3), 
               weight_path = None):
    
    model = Sequential()

    model.add(BatchNormalization(input_shape = (256, 256, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(256, 256, 3)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))
    
    if(weight_path!=None):
        if os.path.isfile(weight_path):
            model.load_weights(weight_path)
    
    return model

In [19]:
# model.compile(optimizer = "adam",
#               loss='binary_crossentropy',
#               metrics=["accuracy", fbeta_score_K])

In [20]:
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

In [ ]:
# adding one step ensure that all of the data will be passed through
print(train_generator.n//train_generator.batch_size + 1)
print(valid_generator.n//train_generator.batch_size + 1)
print(valid_generator.n//train_generator.batch_size + 1)

In [ ]:
# model.fit_generator(train_generator, 
#                     steps_per_epoch = (train_generator.n//train_generator.batch_size + 1),
#                     validation_data = valid_generator,
#                     validation_steps = (valid_generator.n//valid_generator.batch_size + 1),
# #                     callbacks = early_stopping,
#                     epochs = 2,
#                     use_multiprocessing = True,
#                     verbose = 1)

### Validation Predictions

In [ ]:
# trying out validation predictions so I can score validation set.
test_generator.reset()
validation_pred = model.predict_generator(valid_generator,
                                          steps = (valid_generator.n//valid_generator.batch_size + 1),
                                          verbose = 1)

In [ ]:
# np.save("./aws_raw_preds/sequential_val_preds", arr = validation_pred)

In [ ]:
validation_predictions = predictions_to_tags(validation_pred, label_columns, 5e-4)

In [ ]:
val_results = pd.DataFrame(validation_predictions, columns = ['tags'])
val_results.insert(0, column = "image_name", value = valid_generator.filenames)
val_results['image_name'] = "train_" + val_results['image_name'].map(lambda x: x.strip("./kaggle_tif_data/train-tif-v2/"))
val_results.head()

In [ ]:
def f2_score(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')


**F-beta score on Validation Data**

### Test Predictions

In [ ]:
# PREDICTION

# need to reset the test generator before every prediction output
test_generator.reset()

# test predictions
pred = model.predict_generator(test_generator,
steps = test_generator.n//test_generator.batch_size,
verbose = 1)

### Save predictions to Array to export from AWS

In [ ]:
# np.save("./aws_raw_preds/sequential_test_preds", arr = pred)

## Beefier CNN

In [68]:
def bigger_cnn(input_shape = (256, 256, 3), weights = None):
    model = Sequential()
    model.add(BatchNormalization(input_shape = (256, 256, 3)))

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))
    
    if(weight_path!=None):
        if os.path.isfile(weight_path):
            model.load_weights(weight_path)
    
    return model

In [ ]:
cnn.compile(optimizer = "adam",
              loss='binary_crossentropy',
              metrics=["accuracy"])

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

In [ ]:
cnn.fit_generator(train_generator, 
                  steps_per_epoch = train_generator.n//train_generator.batch_size,
                  validation_data = valid_generator,
                  validation_steps = valid_generator.n//valid_generator.batch_size,
#                     callbacks = early_stopping,
                  epochs = 1,
                  use_multiprocessing = True,
                  verbose = 1)

In [ ]:
# PREDICTION

# need to reset the test generator before every prediction output
test_generator.reset()

# test predictions
cnn_pred = cnn.predict_generator(test_generator,
                             steps = test_generator.n//test_generator.batch_size,
                             verbose = 1)

In [ ]:
cnn_pred[0]

## Resnet 50

In [ ]:
def resnet50(input_shape = (256, 256, 3), weights = None):
    from keras.model.resnet50 import ResNet50
    base_resnet = ResNet50(include_top = False, 
                           weights = 'imagenet',  
                           pooling = None, 
                           classes = 17)

    model = Sequential()
    model.add(BatchNormalization(input_shape = input_shape))
    model.add(base_resnet)
    model.add(Flatten())
    model.add(Dense(17, activation='sigmoid'))
   
    if(weight_path!=None):
        if os.path.isfile(weight_path):
            model.load_weights(weight_path)
    return model

# Modeling Steps


### train the model
#### Do early stopping - record number epochs
### Score on val
### Submit a prediction
#### training the model on full dataset
### Submit a prediction
"

### Model Prediction and Evaluation

In [ ]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

y_true, y_pred = np.round(np.random.rand(100, 3)), np.round(np.random.rand(100, 3))
# ensure, that y_true has at least one 1, because sklearn's fbeta can't handle all-zeros
y_true[:, 0] += 1 - y_true.sum(axis=1).clip(0, 1)

fbeta_keras = fbeta(K.variable(y_true), K.variable(y_pred)).eval(session=K.get_session())
fbeta_sklearn = fbeta_score(y_true, np.round(y_pred), beta=2, average='samples')

print('Scores are {:.3f} (sklearn) and {:.3f} (keras)'.format(fbeta_sklearn, fbeta_keras))

# Optimize fbeta thresholds

In [ ]:
# import numpy as np
# from sklearn.metrics import fbeta_score

# def optimise_f2_thresholds(y, p, verbose=True, resolution=100):
#     def mf(x):
#         p2 = np.zeros_like(p)
#         for i in range(17):
#             p2[:, i] = (p[:, i] > x[i]).astype(np.int)
#             score = fbeta_score(y, p2, beta=2, average='samples')
#         return score

#     x = [0.2]*17
#     for i in range(17):
#     best_i2 = 0
#     best_score = 0
#     for i2 in range(resolution):
#       i2 /= resolution
#       x[i] = i2
#       score = mf(x)
#       if score > best_score:
#         best_i2 = i2
#         best_score = score
#     x[i] = best_i2
#     if verbose:
#       print(i, best_i2, best_score)

#   return x